In [1]:
import re
from pycparser import c_parser, c_ast
import pandas as pd
from io import StringIO
from google.colab import files

# Upload the processed CSV file
uploaded = files.upload()  # Upload your file

# Load dataset
file_path = list(uploaded.keys())[0]  # Get the uploaded filename dynamically
df = pd.read_csv(file_path)

def tokenize_code(code):
    if pd.isna(code) or not isinstance(code, str):
        return []

    # Remove comments
    code = re.sub(r'//.*?|/\*.*?\*/', '', code, flags=re.S).strip()

    # Ensure it's a valid C snippet by wrapping it in a basic function
    wrapped_code = f"void dummy(){{\n{code}\n}}"

    parser = c_parser.CParser()
    try:
        ast = parser.parse(wrapped_code)
        return extract_tokens(ast)
    except Exception:
        # Fallback to basic tokenization using regex
        return re.findall(r'\w+|\S', code)

def extract_tokens(ast):
    tokens = []

    class TokenVisitor(c_ast.NodeVisitor):
        def __init__(self):
            self.tokens = []

        def visit(self, node):
            if isinstance(node, c_ast.ID):
                self.tokens.append(node.name)
            self.generic_visit(node)

    visitor = TokenVisitor()
    visitor.visit(ast)

    return visitor.tokens

# Apply tokenization to dataset
df['Tokens'] = df['Faulty Code'].apply(tokenize_code)

# Save the processed DataFrame to a CSV file
output_filename = "tokenized_dataset.csv"
df.to_csv(output_filename, index=False)

# Download the file
files.download(output_filename)


ModuleNotFoundError: No module named 'google'